In [1]:
import os, sys
import numpy as np
import json
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam, SGD
import tqdm

## Prepare the data --> one hot encoding matrices

In [2]:
BATCH_SIZE = 256
random.seed(42)

In [3]:
aptamer_dataset_file = "../data/aptamer_dataset.json"

def construct_dataset():
    with open(aptamer_dataset_file, 'r') as f:
        aptamer_data = json.load(f)
    full_dataset = []
    for aptamer in aptamer_data:
        peptides = aptamer_data[aptamer]
        for peptide, _ in peptides:
            peptide = peptide.replace("_", "")
            if len(aptamer) == 40 and len(peptide) == 8:
                full_dataset.append((aptamer, peptide))
    return full_dataset

In [4]:
full_dataset = construct_dataset()
random.shuffle(full_dataset)
training_set = full_dataset[:int(0.8*len(full_dataset))]
test_set = full_dataset[int(0.8*len(full_dataset)):]

In [5]:
class TrainDataset(torch.utils.data.Dataset):
    def __init__(self, training_set):
        super(TrainDataset, self).__init__() 
        self.training_set = training_set
        n = len(training_set)
        self.training_set = training_set[:n-n%BATCH_SIZE]
        
    def __len__(self):
        return len(self.training_set)

    def __getitem__(self, idx):
        aptamer, peptide = self.training_set[idx]
        return aptamer, peptide
    
class TestDataset(torch.utils.data.Dataset):
    def __init__(self, test_set):
        super(TestDataset, self).__init__() 
        self.test_set = test_set
        n = len(test_set)
        self.test_set = test_set[:n-n%BATCH_SIZE]
        
    def __len__(self):
        return len(self.test_set)

    def __getitem__(self, idx):
        aptamer, peptide = self.test_set[idx]
        return aptamer, peptide

In [6]:
train_dataset = TrainDataset(training_set)
test_dataset = TestDataset(test_set)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE)

In [7]:
aa_list = ['R', 'L', 'S', 'A', 'G', 'P', 'T', 'V', 'N', 'D', 'C', 'Q', 'E', 'H', 'I', 'K', 'M', 'F', 'W', 'Y']
na_list = ['A', 'C', 'G', 'T']

## Takes a peptide and aptamer sequence and converts to one-hot matrix
def one_hot(sequence_list, seq_type='peptide'):
    if seq_type == 'peptide':
        letters = aa_list
    else:
        letters = na_list
    
    one_hot = np.zeros((len(sequence_list), len(sequence_list[0]), len(letters)))
    for j in range(len(sequence_list)):
        sequence = sequence_list[j]
        for i in range(len(sequence)):
            element = sequence[i]
            idx = letters.index(element)
            one_hot[j][i][idx] = 1
    return one_hot

## Model --> CNN

In [8]:
class TwoLayer(nn.Module):
    def __init__(self):
        super(TwoLayer, self).__init__()
        self.linear_apt_1 = nn.Linear(40, 40)
        self.linear_apt_2 = nn.Linear(40, 1)
        
        self.linear_pep_1 = nn.Linear(8, 8)
        self.linear_pep_2 = nn.Linear(8, 1)
        
        self.relu = nn.ReLU()
    
        self.sequential_pep = nn.Sequential(self.linear_pep_1,
                                            self.relu,
                                            self.linear_pep_2)
        
        self.sequential_apt = nn.Sequential(self.linear_apt_1,
                                            self.relu,
                                            self.linear_apt_2)
                
    def forward(self, apt, pep):
        apt = self.sequential_apt(apt)
        pep = self.sequential_pep(pep)
        apt = apt.view(-1, 1).T
        pep = pep.view(-1, 1).T
        x = torch.cat((apt, pep), 1)
        x = F.sigmoid(x)
        return x
    
    def loss(self, prediction, label):
        l = nn.MSELoss()
        label = torch.FloatTensor(label)
        label = label.reshape((1, 1))
        return l(torch.FloatTensor(prediction), label)

In [9]:
class ConvNet(nn.Module):
    def __init__(self, batch_size):
        super(ConvNet, self).__init__()
        self.cnn_apt_1 = nn.Conv2d(BATCH_SIZE, 40, 1)
        self.cnn_apt_2 = nn.Conv2d(40, 10, 1)
        self.cnn_apt_3 = nn.Conv2d(10, 1, 1)
        self.fc_apt_1 = nn.Linear(160, 1)
        
        self.cnn_pep_1 = nn.Conv2d(BATCH_SIZE, 8, 1)
        self.cnn_pep_2 = nn.Conv2d(8, 1, 1)
        self.fc_pep_1 = nn.Linear(64, 1)
        
        self.pool = nn.MaxPool2d(2, 1)
        self.relu = nn.ReLU()
        
        #self.dropout = nn.Dropout(0.1)
        
        self.sequential_pep = nn.Sequential(self.cnn_pep_1,
                                            #self.dropout,
                                            self.relu, 
                                            self.pool, 
                                            self.cnn_pep_2)
        
        self.sequential_apt = nn.Sequential(self.cnn_apt_1, 
                                            #self.dropout,
                                            self.relu, 
                                            self.pool, 
                                            self.cnn_apt_2, 
                                            #self.dropout,
                                            self.relu, 
                                            self.pool, 
                                            self.cnn_apt_3)
        
        self.fc1 = nn.Linear(209, BATCH_SIZE)
        
    def forward(self, apt, pep):
        apt = self.sequential_apt(apt)
        pep = self.sequential_pep(pep)
        
        apt = apt.view(-1, 1).T
        pep = pep.view(-1, 1).T
        
        x = torch.cat((apt, pep), 1)
        x = self.fc1(x)
        x = torch.sigmoid(x)
        return x
    
    def loss(self, prediction, label):
        l = nn.MSELoss()
        label = torch.FloatTensor(label)
        return l(torch.FloatTensor(prediction), label)

In [10]:
model = ConvNet(batch_size=BATCH_SIZE)
def weights_init(m):
    if isinstance(m, nn.Conv2d):
        nn.init.xavier_uniform_(m.weight.data)
        nn.init.zeros_(m.bias.data)
    if isinstance(m, nn.Linear):
        nn.init.kaiming_uniform_(m.weight.data, nonlinearity='relu')
        nn.init.zeros_(m.bias.data)

#model_2 = TwoLayer()
# def weights_init_2(m):
#     if isinstance(m, nn.Linear):
#         nn.init.xavier_uniform_(m.weight.data)
#         nn.init.zeros_(m.bias.data)

model.apply(weights_init)
optimizer = Adam(model.parameters(), lr=1e-3, weight_decay=1e-2)

In [11]:
# Training Loop
for epoch in range(1):
    print("Epoch: ", epoch)
    model.train()
    running_loss = 0.0
    # Come up with a trainloader
    for batch_idx, (aptamer, peptide) in enumerate(tqdm.tqdm(train_loader)):
        # Peptide and aptamer, one-hot encode them 
        pep = one_hot(peptide, seq_type='peptide')
        apt = one_hot(aptamer, seq_type='aptamer')
        
        pep = torch.FloatTensor(np.reshape(pep, (1, pep.shape[0], pep.shape[1], pep.shape[2])))
        apt = torch.FloatTensor(np.reshape(apt, (1, apt.shape[0], apt.shape[1], apt.shape[2])))
        
        output = model(apt, pep)
        
        label = np.ones((1, BATCH_SIZE))
        loss = model.loss(output, label)
        optimizer.zero_grad()
        loss.backward()
        
        #hyperparameter
        clip = 5
        torch.nn.utils.clip_grad_norm(model.parameters(), clip)
        
        optimizer.step()
        running_loss += loss.item()
        if batch_idx % 10 == 9:
            print('[%d, %5d] loss: %.7f' %
                  (epoch + 1, batch_idx + 1, running_loss / 10*BATCH_SIZE))
            running_loss = 0.0
    
print('Finished Training')

  0%|          | 1/2422 [00:00<04:58,  8.10it/s]

Epoch:  0


  0%|          | 11/2422 [00:00<03:42, 10.84it/s]

[1,    10] loss: 69.7710876


  1%|          | 21/2422 [00:01<02:58, 13.46it/s]

...

[1,  2410] loss: 0.4722994


100%|█████████▉| 2420/2422 [07:20<00:00,  4.58it/s]

[1,  2420] loss: 0.4621632


100%|██████████| 2422/2422 [07:20<00:00,  4.34it/s]

Finished Training


## Evaluation --> compare to random

In [14]:
correct = 0
incorrect = 0
for batch_idx, (aptamer, peptide) in enumerate(tqdm.tqdm(test_loader)):
    pep = one_hot(peptide, seq_type='peptide')
    apt = one_hot(aptamer, seq_type='aptamer')
    
    pep = torch.FloatTensor(np.reshape(pep, (1, pep.shape[0], pep.shape[1], pep.shape[2])))
    apt = torch.FloatTensor(np.reshape(apt, (1, apt.shape[0], apt.shape[1], apt.shape[2])))

    output = model(apt, pep).detach().numpy().flatten()
    for i in range(output.shape[0]):
        o = output[i]
        print(o)
        if o > 0.9:
            correct += 1
        else:
            incorrect += 1
    break

print('Recall of the network on the test samples: %d %%' % (100* correct/(correct + incorrect)))

  0%|          | 0/605 [00:00<?, ?it/s]

0.9567549
0.957085
0.9568714
0.957408
0.95679474
0.95700353
0.95702887
0.95687366
0.9571973
0.9572274
0.9568822
0.9568617
0.9568713
0.95701253
0.9574633
0.9573711
0.9573384
0.9572117
0.9570139
0.9570279
0.9568444
0.9571089
0.9572363
0.95738965
0.9571937
0.95730424
0.95736265
0.9568217
0.9568214
0.9570708
0.9569886
0.95685804
0.95681494
0.95696676
0.957396
0.95748353
0.9572729
0.95731765
0.9570516
0.95701504
0.95679146
0.9572955
0.9571367
0.9570932
0.95738256
0.95700616
0.9568899
0.9572897
0.9570907
0.9569316
0.95729786
0.95738834
0.9571122
0.95737904
0.9573532
0.9568454
0.95685476
0.9573713
0.95733887
0.9568479
0.9572363
0.9574307
0.95733786
0.9571064
0.9569708
0.95716006
0.9572149
0.95698816
0.95695084
0.95740706
0.9569526
0.956785
0.9569647
0.956881
0.9569896
0.95706743
0.9568834
0.95719403
0.9568906
0.9569418
0.9568436
0.95743215
0.9575058
0.95702386
0.9568679
0.9568499
0.9568144
0.9572034
0.95740277
0.9575205
0.95686996
0.9567855
0.95737535
0.9572734
0.9571198
0.9574195
0.95708543


# sampling --> need to change

In [13]:
k = 1e5

# Generate uniformly without replacement
def get_samples(kind="pep",num=k):
    if kind == "apt":
        samples = [all_aptamers[i] for i in np.random.choice(len(all_aptamers), num_samples, replace=False)]
    else:
        samples = [all_peptides[i] for i in np.random.choice(len(all_peptides), num_samples, replace=False)]
    return samples

# Sample x' from P_X (assume peptides follow NNK)
def get_x_prime(k):
    x_primes = []
    for _ in range(k):
        pvals = [0.089]*3 + [0.065]*5 + [0.034]*12
        x_idx = np.random.choice(20, 7, p=pvals)
        x_prime = "M"
        for i in x_idx:
            x_prime += aa_list[i]
        x_primes.append(x_prime)
    return x_primes

# Sample y' from P_Y (assume apatamers follow uniform)
def get_y_prime(k):
    y_primes = []
    for _ in range(k):
        y_idx = np.random.randint(0, 4, 40)
        y_prime = ""
        for i in y_idx:
            y_prime += na_list[i]
        y_primes.append(y_prime)
    return y_primes